In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import json
import pandas as pd

In [ ]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get("https://section.blog.naver.com/")

driver.find_element(
    By.CSS_SELECTOR,
    "#header > div.header_common > div > div.area_search > form > fieldset > div > input",
).send_keys("뜨개카페")
driver.find_element(
    By.CSS_SELECTOR,
    "#header > div.header_common > div > div.area_search > form > fieldset > div > input",
).send_keys(Keys.ENTER)

# 페이지 반복
cafe = []
for i in range(1, 2):
    url = f"https://section.blog.naver.com/Search/Post.naver?pageNo={i}&rangeType=ALL&orderBy=recentdate&keyword=%EB%9C%A8%EA%B0%9C%EC%B9%B4%ED%8E%98"
    driver.get(url)
    time.sleep(2)

    blog_links = []

    # 한페이지에 7개 블로그 글
    blog_cards = driver.find_elements(
        By.CSS_SELECTOR, "#content > section > div.area_list_search > div"
    )[:7]

    # 블로그 주소, id, no 추출
    for card in blog_cards:
        try:
            elem = card.find_element(
                By.CSS_SELECTOR, "div > div.info_post > div.desc > a.desc_inner"
            )
            blog_links.append(elem.get_attribute("href"))
        except:
            continue

    for blog_url in blog_links:
        try:
            blogID = blog_url.split("/")[-2]
            logNo = blog_url.split("/")[-1]
            # 블로그 접속
            driver.get(
                f"https://blog.naver.com/PostView.naver?blogId={blogID}&logNo={logNo}"
            )
            time.sleep(2)

            try:
                # 지도 정보가 있는지 확인
                map_elem = driver.find_element(
                    By.CSS_SELECTOR, "div.se-module.se-module-map-text > a"
                )
                map_data_raw = map_elem.get_attribute("data-linkdata")
                map_data = json.loads(map_data_raw)
                # 블로그 내용 크롤링
                blog = driver.find_element(
                    By.CSS_SELECTOR, "#printPost1 > tbody > tr > td.bcc"
                ).text

                cafe.append(
                    {
                        "logNo": logNo,
                        "카페이름": map_data["name"],
                        "카페주소": map_data["address"],
                        "위도": map_data["latitude"],
                        "경도": map_data["longitude"],
                        "블로그내용": blog,
                    }
                )

            except:
                print("지도정보 없음")

        except Exception as e:
            print("오류 발생:", e)
            continue

지도정보 없음
지도정보 없음
지도정보 없음
지도정보 없음


In [3]:
cafe

[{'logNo': '223945383193',
  '카페이름': '뉴뉴 명동점',
  '카페주소': '서울특별시 중구 명동4길 22 1-4F',
  '위도': '37.5627008',
  '경도': '126.9837432',
  '블로그내용': '일상\n후암동 근처 나들이\n청록 ・ 3분 전\nURL 복사 이웃추가\n본문 기타 기능\n친구랑 후암동~해방촌 근처에서 놀았는데 역대급 재밌었어서 기록을 남긴다.\n근데 만난건 명동에서 만났다.\n일단 딤딤섬에 가서 딤섬 부수기 함\n크리스피창펀 새우하가우 돼지고기시우마이 가지딤섬 < 이 네개중에 하나도 빼먹어선 안돼 기억해\n방금 만든 크.창은 세상을 다 가진 맛이다\n50m\n© NAVER Corp.\n뉴뉴 명동점\n서울특별시 중구 명동4길 22 1-4F\n그리고 여기를 갔다\n4층짜리 매장에 악세사리랑 잡화들 엄청 많아서 눈이 돌아갔고 양말이랑 키링을 하나씩 샀다\n사진이 업슴..\n쇼핑해서 즐거워진 마음으로 본 목적인 그릇을 사러 이동했다\n버스타고 30분 정도 이동해서 갔는데 이 동네 언덕 쩐다\n아니 근데 어떻게 동네 이름이 후암? ㅋㅋㅋ\n후암~ ㅋㅋㅋㅋ\n50m\n© NAVER Corp.\n도기야\n서울특별시 용산구 두텁바위로58길 7 장우빌딩 1층 1호\n목적지는 이 곳\n이런 일본식그릇을 잔뜩 구경할 수 있었다\n옛~날에 행궁동 소품샵에서 사려다가 참았던 그릇을 거의 반값에 팔고 있길래 운명을 느끼고 구매\n50m\n© NAVER Corp.\n시논샵\n서울특별시 용산구 두텁바위로60길 49 대원빌딩 신관 101호\n10분쯤 언덕을 지나서 근처 시논샵으로 이동\n도기야랑은 다른 감성으로 개이쁜 그릇들을 판매하고 있었다\n눈돌아가서 혼났다 진ㅉ ㅏ다이쁨 가격도 나쁘지않았다\n이 존나 예쁜 그릇을 보고 고민하다가 파스타볼 구매 ㅋㅋ\n머그도 개이쁘지 않은가염\n50m\n© NAVER Corp.\n다다미크레페\n서울특별시 용산구 두텁바위로37길 6 1층 101호\n구매 완 하고 카페갈 겸 안먹어본 크레페를

In [15]:
data = driver.find_element(By.CSS_SELECTOR, "span.se_publishDate.pcol2")
data.text

'2시간 전'

In [ ]:
# 작성일 가져오기 

In [19]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get("https://section.blog.naver.com/")

driver.find_element(
    By.CSS_SELECTOR,
    "#header > div.header_common > div > div.area_search > form > fieldset > div > input",
).send_keys("뜨개카페")
driver.find_element(
    By.CSS_SELECTOR,
    "#header > div.header_common > div > div.area_search > form > fieldset > div > input",
).send_keys(Keys.ENTER)

# 페이지 반복
cafe = []
for i in range(11, 15):
    url = f"https://section.blog.naver.com/Search/Post.naver?pageNo={i}&rangeType=ALL&orderBy=recentdate&keyword=%EB%9C%A8%EA%B0%9C%EC%B9%B4%ED%8E%98"
    driver.get(url)
    time.sleep(2)

    blog_links = []

    # 한페이지에 7개 블로그 글
    blog_cards = driver.find_elements(
        By.CSS_SELECTOR, "#content > section > div.area_list_search > div"
    )[:7]

    # 블로그 주소, id, no 추출
    for card in blog_cards:
        try:
            elem = card.find_element(
                By.CSS_SELECTOR, "div > div.info_post > div.desc > a.desc_inner"
            )
            blog_links.append(elem.get_attribute("href"))
        except:
            continue

    for blog_url in blog_links:
        try:
            blogID = blog_url.split("/")[-2]
            logNo = blog_url.split("/")[-1]
            # 블로그 접속
            driver.get(
                f"https://blog.naver.com/PostView.naver?blogId={blogID}&logNo={logNo}"
            )
            time.sleep(2)

            try:
                # 지도 정보가 있는지 확인
                map_elem = driver.find_element(
                    By.CSS_SELECTOR, "div.se-module.se-module-map-text > a"
                )
                map_data_raw = map_elem.get_attribute("data-linkdata")
                map_data = json.loads(map_data_raw)
                # 블로그 내용 크롤링
                blog = driver.find_element(
                    By.CSS_SELECTOR, "#printPost1 > tbody > tr > td.bcc"
                ).text
                # 블로그 작성일 크롤링
                date = driver.find_element(
                    By.CSS_SELECTOR, "span.se_publishDate.pcol2"
                    ).text

                cafe.append(
                    {
                        "logNo": logNo,
                        "카페이름": map_data["name"],
                        "카페주소": map_data["address"],
                        "위도": map_data["latitude"],
                        "경도": map_data["longitude"],
                        "블로그내용": blog,
                        "블로그작성일":date,
                    }
                )

            except:
                print("지도정보 없음")

        except Exception as e:
            print("오류 발생:", e)
            continue

지도정보 없음
지도정보 없음
지도정보 없음
지도정보 없음
지도정보 없음
지도정보 없음
지도정보 없음
지도정보 없음
지도정보 없음
지도정보 없음
지도정보 없음
지도정보 없음
지도정보 없음


In [32]:
data

'2025. 7. 23. 11:26'

In [20]:
cafe

[{'logNo': '223868951318',
  '카페이름': '일룸 프리미엄샵 마포서대문',
  '카페주소': '서울특별시 서대문구 연희로 148',
  '위도': '37.5703179',
  '경도': '126.9344837',
  '블로그내용': '맛도리\n[연희] 보장된 맛집, 녹원쌈밥 (주차 추천)\n깨춤 ・ 22시간 전\nURL 복사 이웃추가\n본문 기타 기능\n👥 추천해요\n인스타 맛집이 아닌 찐맛집이 고플 때\n맛있는 한식을 먹고싶을 때\n경기 근교 맛집을 온 기분을 내고 싶을 때\n👍 총평\n보장된 맛집\n연희 간다면 또 갈 집\n양 많음\n💫 주변에 함께 갈 곳\n바늘이야기 (뜨개 카페)\n연희동 국화빵 (호지차, 자스민 국화빵)\n라이카 시네마 (영화관)\n사러가마트 (연희동 명물 마트)\n비스켓스튜디오 (소품샵)\n등\n연희동 국화빵을 2주간 노래를 부르다\n국화빵만 먹기 뭐해서\n연희동에서 먹기로 했는데\n그간 연희 갈 때면 제이가 노래를 하던\n녹원쌈밥을 드디어 가봤다!\n연희 중심 거리와는 약간 거리가 있지만\n가는 길에 카페 등 볼 게 많아서 걸어가긴 쉬울 듯!\n도착하자 풍겨지는 맛집 분위기\n연식이 보이는 간판과\n주택으로 된 음식점이었다.\n우리는 2시 50분쯤 도착해서 기다리진 않았다!\n브레이크 타임(주문)은 3시부터지만\n그 전에 주문 한 경우 3시 반까지는 먹을 수 있다.\n연희동 추천 리스트도 있고\n은근 아기자기했다\n보이는가\n한식 맛집을 ai 이미지로 생성하면 나올 것 같은 풍경\n심지어 오른쪽 뒤에 있는건 김치냉장고 ㅋㅋㅋ\n우리는 녹원 쌈밥 2인분이랑 메밀전 동동주 한 잔을 시켰다.\n오래된 맛집다운 벽지\n시키고 나서 바로 쌈채소 푸러 갔는데\n위엄이 느껴지는\n쌈채소 종류와 퀄리티!\n이 주변 살면\n부모님이 날 데리고 매주 올 것같은 느낌\nㅋㅋㅋㅋ\n녹원 쌈밥(오징어, 버섯, 제육) 17,000원 /인\n2인분\n삼합은\n간이 안 쎈건 아닌데\n심심한 것도 아닌..\n적당한 간이어서 맛있었는데\n밑

In [31]:
cafe[10:15]

[{'logNo': '223944062628',
  '카페이름': '용다리',
  '카페주소': 'Đ. Nguyễn Văn Linh, Phước Ninh, Hải Châu, Đà Nẵng 550000 베트남',
  '위도': '16.0612215',
  '경도': '108.2296818',
  '블로그내용': "25' 다낭\n[다낭 여행] 다낭 한시장 + 한시장 쇼핑리스트\n여진 ・ 2025. 7. 23. 13:30\nURL 복사 이웃추가\n본문 기타 기능\n위치한시장\n배부르게 먹고 나와서\n한강 구경하기\n강바람이 시원하게 불었다.\n(물론 약간의 습도는 감안....😁)\n멀찍이 용다리도 보였고\n용다리\nĐ. Nguyễn Văn Linh, Phước Ninh, Hải Châu, Đà Nẵng 550000 베트남\n용다리에서는 매주 금,토,일에 불쇼가 열리는데\n여태 한번도 보지못함...😅\n한강 근처에 공원처럼 꾸며진 이곳은\n밤에 와서 보면 야경이 장관일 것 같다 👍\n친구랑 사진 적당히 남기고\n원래의 목적인 한시장으로 향했다.\n대망의 한시장, 두둥두둥\n한시장\n119 Trần Phú, Hải Châu, Đà Nẵng 550000 베트남\n한시장에 정말 다양한 품목들이 판매되고 있지만\n나는 필요한 몇몇 품목만 사기로함!\n한시장 쇼핑리스트\n1. 크록스\n시세표가 카페나 인터넷에 돌아다닐 뿐 아니라\n현지 시장에도 떡하니 걸려있다 😅\n거의 저정도 선에서 구매가 다 이루어지기 때문에\n여러 켤레 사서 할인 받는 것을 추천!\n나는 256번 가게에서\n나는 남자 크록스 기본 1, 샌들 1, 지비츠 4 세트 구매함\n62만동에서 2만동 깎아서 60만동 계산했다.\n더 많이 사거나 금액이 애매하면 조금씩 할인 더 받길\n2. 방도 & 머리핀\n집에 있는 비똥이에 감아주려고 산 방도\n머리핀은 방도 사러갔다가 구경하면서\n친구랑 같은 걸로 맞춰서 샀다.\n방도는 여러 브랜드 짭이 있으니 맞춰서 사고\n에*메스는 15만동 정도라고 한다.\n3. 뜨개가방 

In [ ]:
# sql

In [4]:
import pymysql

In [5]:
envs = {}
with open(".env", "r") as f:
    for line in f.readlines():
        key, value = line.rstrip().split("=")
        envs[key] = value

conn = pymysql.connect(
    host=envs["DB_HOST"],
    port=int(envs["DB_PORT"]),
    user=envs["DB_USER"],
    password=envs["DB_PASSWORD"],
    database=envs["DB_DATABASE"],
    charset="utf8mb4",
)

In [7]:
with conn.cursor() as cur:
    sql = "INSERT INTO tb_cafe (logNo,cafename,cafeaddress,latitude,longitude,blogtext) VALUES (%s,%s,%s,%s,%s,%s)"

    values = [
        (
            data["logNo"],
            data["카페이름"],
            data["카페주소"],
            float(data["위도"]),
            float(data["경도"]),
            data["블로그내용"],
        )
        for data in cafe
    ]

    cur.executemany(sql, values)
conn.commit()

In [16]:
import pymysql.cursors # 필드값 까지 가져오게 됨 

with conn.cursor(cursor=pymysql.cursors.DictCursor) as cur: 
    result = cur.execute('SELECT id, logNo, cafename FROM tb_cafe ORDER BY logNo ASC')
    datas = cur.fetchall()
datas

[{'id': 3, 'logNo': '223944255718', 'cafename': '맨리 비치'},
 {'id': 2, 'logNo': '223944257143', 'cafename': '누뗀'},
 {'id': 1, 'logNo': '223944317831', 'cafename': '미쁘동 연남본점'}]